## Import Libraries

In [52]:
# Data analysis library
import numpy as np
import pandas as pd
import joblib
import seaborn as sns
%matplotlib inline
# Machine Learning library
import sklearn
from sklearn.metrics import roc_curve, auc, accuracy_score, plot_confusion_matrix, plot_roc_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from lightgbm import LGBMClassifier
from lightgbm import plot_importance, plot_metric
from sklearn.metrics import f1_score
# Hyperparameter tunning library
import optuna

# Model experimentation library
import mlflow
import mlflow.lightgbm
from mlflow.tracking import MlflowClient

import pickle
# Plotting library
import matplotlib.pyplot as plt
# Prevent figures from displaying by turning interactive mode off using the function
plt.ioff()
import warnings
warnings.filterwarnings("ignore")

In [53]:
print(f'Numpy version is {np.__version__}')
print(f'Pandas version is {pd.__version__}')
print(f'sklearn version is {sklearn.__version__}')
print(f'joblib version is {joblib.__version__}')
print(f'optuna version is {optuna.__version__}')
print(f'mlflow version is {mlflow.__version__}')

Numpy version is 1.20.2
Pandas version is 1.2.4
sklearn version is 0.24.1
joblib version is 1.0.1
optuna version is 2.7.0
mlflow version is 1.15.0


## Load Dataset


In [54]:
## Files
data_file = 'dataset/train.csv'

# to display all the columns of the dataframe in the notebook
pd.set_option('display.max_columns', None)
# Load train loan dataset 
try:
    df = pd.read_csv(data_file)
    print("The dataset has {} samples with {} features.".format(*df.shape))
except:
    print("The dataset could not be loaded. Is the dataset missing?")

The dataset has 532428 samples with 45 features.


In [55]:
df.head()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,batch_enrolled,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,pymnt_plan,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,verification_status_joint,last_week_pay,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,loan_status
0,58189336,14350,14350,14350.0,36 months,,19.19,E,E3,clerk,9 years,OWN,28700.0,Source Verified,n,NaN,debt_consolidation,Debt consolidation,349xx,FL,33.88,0.0,1.0,50.0,75.0,14.0,1.0,22515.0,73.1,28.0,f,1173.84,0.0,0.0,0.0,0.0,74.0,INDIVIDUAL,NaN,26th week,0.0,0.0,28699.0,30800.0,0
1,70011223,4800,4800,4800.0,36 months,BAT1586599,10.99,B,B4,Human Resources Specialist,< 1 year,MORTGAGE,65000.0,Source Verified,n,NaN,home_improvement,Home improvement,209xx,MD,3.64,0.0,1.0,NaN,NaN,6.0,0.0,7624.0,23.2,13.0,w,83.95,0.0,0.0,0.0,0.0,NaN,INDIVIDUAL,NaN,9th week,0.0,0.0,9974.0,32900.0,0
2,70255675,10000,10000,10000.0,36 months,BAT1586599,7.26,A,A4,Driver,2 years,OWN,45000.0,Not Verified,n,NaN,debt_consolidation,Debt consolidation,447xx,OH,18.42,0.0,0.0,NaN,NaN,5.0,0.0,10877.0,31.2,19.0,w,56.47,0.0,0.0,0.0,0.0,NaN,INDIVIDUAL,NaN,9th week,0.0,65.0,38295.0,34900.0,0
3,1893936,15000,15000,15000.0,36 months,BAT4808022,19.72,D,D5,Us office of Personnel Management,10+ years,RENT,105000.0,Not Verified,n,> My goal is to obtain a loan to pay off my hi...,debt_consolidation,Debt consolidation,221xx,VA,14.97,0.0,2.0,46.0,NaN,10.0,0.0,13712.0,55.5,21.0,f,4858.62,0.0,0.0,0.0,0.0,NaN,INDIVIDUAL,NaN,135th week,0.0,0.0,55564.0,24700.0,0
4,7652106,16000,16000,16000.0,36 months,BAT2833642,10.64,B,B2,LAUSD-HOLLYWOOD HIGH SCHOOL,10+ years,RENT,52000.0,Verified,n,NaN,credit_card,refi,900xx,CA,20.16,0.0,0.0,NaN,NaN,11.0,0.0,35835.0,76.2,27.0,w,2296.41,0.0,0.0,0.0,0.0,NaN,INDIVIDUAL,NaN,96th week,0.0,0.0,47159.0,47033.0,0


## Tracking Experiments

### Tracking  Metadata 

MLflow supports two types of backend stores: *file store* and *database-backed* store.

- Local file path (specified as file:/my/local/dir), where data is just directly stored locally. Defaults to `mlruns/`
- Database encoded as <dialect>+<driver>://<username>:<password>@<host>:<port>/<database>. Mlflow supports the dialects mysql, mssql, sqlite, and postgresql. For more details, see SQLAlchemy database uri.
- HTTP server (specified as https://my-server:5000), which is a server hosting an MLFlow tracking server.
 
**For this Demo we are usking Sqlite as a Database to store metadata.**

### Artifact stores
- Amazon S3
- Azure Blob Storage
- Google Cloud Storage
- FTP server
- SFTP Server
- NFS
- HDFS
    
**We are using local mlrun folder to store all required artifacts.**

Base Syntax to start mlflow as a service

```
mlflow server \
    --backend-store-uri /mnt/persistent-disk \
    --default-artifact-root s3://my-mlflow-bucket/ \
    --host 0.0.0.0
    --port 5000
```
![alt text](images/localhost_sqlite.png "Title")
**Note:** To start the service for this experiment copy below command and execute it in Anaconda prompt
```bash
mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root mlruns/ --host 127.0.0.1 --port 5000
```

In [56]:
experiment_name = "Loan deaulter predictionv1"
artifact_repository = './mlflow'

# Provide uri and connect to your tracking server
mlflow.set_tracking_uri('http://127.0.0.1:5000/')

# Initialize client
client = MlflowClient()

# If experiment doesn't exist then it will create new
# else it will take the experiment id and will use to to run the experiments
try:
    # Create experiment 
    experiment_id = client.create_experiment(experiment_name, artifact_location=artifact_repository)
except:
    # Get the experiment id if it already exists
    experiment_id = client.get_experiment_by_name(experiment_name).experiment_id

## Separate dataset into train and test

In [57]:
# Let's separate into train and test set
# Remember to seet the seed (random_state for this sklearn function)

X_train, X_test, y_train, y_test = train_test_split(df, df.loan_status,
                                                    test_size=0.1,
                                                    random_state=42) # we are setting the seed here
X_train.shape, X_test.shape

((479185, 45), (53243, 45))


**Remember: the aim of this demo and this particular project is to show you how use mlflow for tracking and put models in production. Surely there are additional things you can do on this dataset, to extract additional value from the features.**

**In order to capitalise on the deployment aspect of things, we deliberately kept the engineering side simple, yet include many of the traditional engineering steps, so you get a full flavour of building and deploying a machine learning model.**

## Handling Missing Value as NaN

In [58]:
# converting blank value to NaN value.
X_train = X_train.replace(' ', np.nan)
X_test = X_test.replace(' ', np.nan)

#ratio of null values
X_train.isnull().sum()/X_train.shape[0] *100

member_id                       0.000000
loan_amnt                       0.000000
funded_amnt                     0.000000
funded_amnt_inv                 0.000000
term                            0.000000
batch_enrolled                 35.907635
int_rate                        0.000000
grade                           0.000000
sub_grade                       0.000000
emp_title                       5.787952
emp_length                      5.044816
home_ownership                  0.000000
annual_inc                      0.000626
verification_status             0.000000
pymnt_plan                      0.000000
desc                           85.810073
purpose                         0.000000
title                           0.017321
zip_code                        0.000000
addr_state                      0.000000
dti                             0.000000
delinq_2yrs                     0.002713
inq_last_6mths                  0.002713
mths_since_last_delinq         51.149139
mths_since_last_

In [71]:
# to create a file as pickle
def picklesave(filename,file):
    with open(filename, 'wb') as handle:
        pickle.dump(file, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Converting into numerical feature and New feature

In [60]:
emp_len_dict = {'< 1 year':0,'1 year':1,'2 years':2,'3 years':3,'4 years':4,'5 years':5,'6 years':6,'7 years':7,'8 years':8,'9 years':9,'10+ years':10}
X_train['emp_length'] = X_train['emp_length'].map(emp_len_dict)
X_test['emp_length'] = X_test['emp_length'].map(emp_len_dict)

picklesave('artifacts/emp_len_dict.pickle', emp_len_dict)

# create the temporal var
X_train['Long_emp_length'] = X_train['emp_length'].apply(lambda x : 'Yes' if x==10 else 'No')
X_test['Long_emp_length'] = X_test['emp_length'].apply(lambda x : 'Yes' if x==10 else 'No')



## Handling Missing categorical values

For categorical variables, we will fill missing information by adding an additional category: "missing"

In [61]:
# make a list of the categorical variables that contain missing values
vars_with_na = [var for var in X_train.columns if X_train[var].dtypes=='O']

# print the variable name and the percentage of missing values
for var in vars_with_na:
    print(var, np.round(X_train[var].isnull().mean(), 3),  ' % missing values')

term 0.0  % missing values
batch_enrolled 0.359  % missing values
grade 0.0  % missing values
sub_grade 0.0  % missing values
emp_title 0.058  % missing values
home_ownership 0.0  % missing values
verification_status 0.0  % missing values
pymnt_plan 0.0  % missing values
desc 0.858  % missing values
purpose 0.0  % missing values
title 0.0  % missing values
zip_code 0.0  % missing values
addr_state 0.0  % missing values
initial_list_status 0.0  % missing values
application_type 0.0  % missing values
verification_status_joint 0.999  % missing values
last_week_pay 0.0  % missing values
Long_emp_length 0.0  % missing values


In [62]:
# function to replace NA in categorical variables
for col in ["emp_title","title"]:
    X_train[col].fillna('Missing',inplace=True)
    X_test[col].fillna('Missing',inplace=True)
    
X_train[vars_with_na].isnull().sum()

## Handling Missing Numerical values

In [64]:
mean_var_dict = {}
impute_col = ['emp_length','annual_inc','delinq_2yrs','inq_last_6mths','open_acc','pub_rec','revol_util','total_acc','collections_12_mths_ex_med','acc_now_delinq','tot_coll_amt','tot_cur_bal','total_rev_hi_lim']
# Imputing with Mean
#impute_col = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
for col in impute_col:
    # calculate the mean
    mean_val = X_train[col].mean()
    
    X_train[col].fillna(mean_val, inplace=True)
    X_test[col].fillna(mean_val, inplace=True)
    # we persist the mean in the dictionary
    mean_var_dict[col] = mean_val
# we save the dictionary for later
picklesave('artifacts/mean_var_dict.pickle', mean_var_dict)
#print(mean_var_dict)
X_train[impute_col].isnull().sum()

emp_length                    0
annual_inc                    0
delinq_2yrs                   0
inq_last_6mths                0
open_acc                      0
pub_rec                       0
revol_util                    0
total_acc                     0
collections_12_mths_ex_med    0
acc_now_delinq                0
tot_coll_amt                  0
tot_cur_bal                   0
total_rev_hi_lim              0
dtype: int64

## Handling Rare values 

In [65]:
# define your threshold here

threshlold = 0.0005
frequent_labels_dict = {}

categorical_col = ["zip_code",'last_week_pay','emp_title']

for variable in categorical_col:
    # locate all the categories that are not rare.
    counts = X_train.groupby([variable])[variable].count() / len(X_train)
    frequent_labels = [x for x in counts.loc[counts>threshlold].index.values]
    frequent_labels_dict[variable] = frequent_labels

    # change the rare category names with the word rare, and thus encoding it.
    X_train[variable] = np.where(X_train[variable].isin(frequent_labels), X_train[variable], 'Rare')
    X_test[variable] = np.where(X_test[variable].isin(frequent_labels), X_test[variable], 'Rare')

# now we save the dictionary
picklesave('artifacts/FrequentLabels.pickle', frequent_labels_dict)

## Feature Selection

In [66]:
exclude_feature = ['loan_amnt','funded_amnt','title','batch_enrolled','member_id','loan_status','desc','mths_since_last_delinq','mths_since_last_record','mths_since_last_major_derog','verification_status_joint']
# Define Target columns
#target = data['loan_status']

# Define numeric and categorical features
numeric_columns = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_columns = X_train.select_dtypes(include=['object']).columns.tolist()
numeric_features = [col for col in numeric_columns if col not in exclude_feature]
categorical_features = [col for col in categorical_columns if col not in exclude_feature]

# Define final feature list for training and validation
features = numeric_features + categorical_features
# Final data for training and validation
X_train = X_train[features]
X_train = X_train.fillna(0)
# now we save the selected list of features
pd.Series(features).to_csv('artifacts/selected_features.csv', index=False)

## Categorical feature Encoding

In [67]:
# Split data in train and validation
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

label_dict = {}


# Perform label encoding for categorical variable
for feature in categorical_features:
    le = LabelEncoder()
    X_train[feature]=  X_train[feature].astype('str')
    #test_data[feature]=  test_data[feature].astype('str')
    X_valid[feature]=  X_valid[feature].astype('str')
    X_test[feature]=  X_test[feature].astype('str')
    le.fit(X_train.loc[:, feature])
    X_train.loc[:, feature] = le.transform(X_train.loc[:, feature])
    X_valid.loc[:, feature] = le.transform(X_valid.loc[:, feature])
    X_test.loc[:, feature] = le.transform(X_test.loc[:, feature])
    label_dict[feature] = le

picklesave("artifacts/labelEncoder.pickle",label_dict)

![alt text](images/MLflow_Model_experimentation.png "Title")

**source: vivek kumar [YouTube]**

### Model Training and Tracking using MLflow

In [68]:
def model_training_tracking(params):
    with mlflow.start_run(experiment_id=experiment_id, run_name='Lightgbm_model1') as run:
        # Get run id 
        run_id = run.info.run_uuid

        # Set the notes for the run
        MlflowClient().set_tag(run_id,
                               "mlflow.note.content",
                               "This is experiment for hyperparameter optimzation for lightgbm models for the Load Defaulter prediction")

        # Define customer tag
        tags = {"Application": "Loan Defaulter Monitoring",
                "release.version": "1.0.0"}

        # Set Tag
        mlflow.set_tags(tags)

        # Log python environment details
        mlflow.log_artifact('requirements.txt')
        # Log selected feature and feature enginering data 
        mlflow.log_artifact('artifacts/selected_features.csv')      
        mlflow.log_artifact('artifacts/FrequentLabels.pickle')      
        mlflow.log_artifact('artifacts/mean_var_dict.pickle')      
        mlflow.log_artifact('artifacts/emp_len_dict.pickle')       
        mlflow.log_artifact("artifacts/labelEncoder.pickle")

        # logging params
        mlflow.log_params(params)
        auc_list = []
        mlflow.lightgbm.autolog()
        lgb_clf = LGBMClassifier(**params)
        lgb_clf.fit(X_train, y_train, 
                    eval_set = [(X_train, y_train), (X_valid, y_valid)], 
                    early_stopping_rounds=100,
                    verbose=20)
        # Log model artifacts
        mlflow.sklearn.log_model(lgb_clf, "model")
        
        # Perform model evaluation 
        lgb_valid_prediction = lgb_clf.predict_proba(X_valid)[:, 1]
        fpr, tpr, thresholds = roc_curve(y_valid, lgb_valid_prediction)
        roc_auc = auc(fpr, tpr) # compute area under the curve
        print("=====================================")
        print("Validation AUC:{}".format(roc_auc))
        print("=====================================")   

        score_lgb_valid_prediction = lgb_clf.predict(X_valid)
        score = f1_score(y_valid, score_lgb_valid_prediction, average='weighted')# compute f1 score
        score = 100* score
        print("=====================================")
        print("Validation f1 score:{}".format(score))
        print("=====================================")  
        
        # log metrics
        mlflow.log_metrics({"Validation_AUC": roc_auc,"Validation_f1": score})
        
        # Plot and save feature importance details
        ax = plot_importance(lgb_clf, height=0.4)
        filename = './images/lgb_validation_feature_importance.png'
        plt.savefig(filename)
        # log model artifacts
        mlflow.log_artifact(filename)

        ax = plot_metric(lgb_clf.evals_result_)
        filename = './images/lgb_validation_metrics_comparision.png'
        plt.savefig(filename)
        # log model artifacts
        mlflow.log_artifact(filename)

        # Plot and save metrics details    
        plot_confusion_matrix(lgb_clf, X_valid, y_valid, 
                              display_labels=['Placed', 'Not Placed'],
                              cmap='magma')
        plt.title('Confusion Matrix')
        filename = './images/lgb_validation_confusion_matrix.png'
        plt.savefig(filename)
        # log model artifacts
        mlflow.log_artifact(filename)

        # Plot and save AUC details  
        plot_roc_curve(lgb_clf, X_valid, y_valid, name='Validation')
        plt.title('ROC AUC Curve')
        filename = './images/lgb_validation_roc_curve.png'
        plt.savefig(filename)
        # log model artifacts
        mlflow.log_artifact(filename)
        return roc_auc

In [69]:
def objective(trial):

    param = {
        "objective": "binary",
        "metric": "auc",
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
        "subsample": trial.suggest_float("subsample", 0.4, 1.0),
        "random_state": 42,
        "boosting_type": "gbdt",
        #"max_depth": 10
    }
    
    score = model_training_tracking(param)
    return score

In [70]:
# Create a study object and optimize the objective function.
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=40)
trial = study.best_trial
print('score: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2021-04-18 14:25:43,841] A new study created in memory with name: no-name-85b0bee1-836e-4847-8de8-b270e7de7f3d
2021/04/18 14:25:44 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of lightgbm. If you encounter errors during autologging, try upgrading / downgrading lightgbm to a supported version, or try upgrading MLflow.


Training until validation scores don't improve for 100 rounds
[20]	training's auc: 0.81228	valid_1's auc: 0.810512
[40]	training's auc: 0.810725	valid_1's auc: 0.809361
[60]	training's auc: 0.814203	valid_1's auc: 0.812824
[80]	training's auc: 0.817833	valid_1's auc: 0.816397
[100]	training's auc: 0.820986	valid_1's auc: 0.819753
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.820986	valid_1's auc: 0.819753
Validation AUC:0.8197533707276448
Validation f1 score:66.23195524719897


[I 2021-04-18 14:26:16,443] Trial 0 finished with value: 0.8197533707276448 and parameters: {'learning_rate': 0.0020422511327989144, 'colsample_bytree': 0.7848430373096997, 'subsample': 0.410500457288687}. Best is trial 0 with value: 0.8197533707276448.
2021/04/18 14:26:17 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of lightgbm. If you encounter errors during autologging, try upgrading / downgrading lightgbm to a supported version, or try upgrading MLflow.


Training until validation scores don't improve for 100 rounds
[20]	training's auc: 0.823706	valid_1's auc: 0.821301
[40]	training's auc: 0.829392	valid_1's auc: 0.827781
[60]	training's auc: 0.830937	valid_1's auc: 0.829384
[80]	training's auc: 0.836588	valid_1's auc: 0.835101
[100]	training's auc: 0.844873	valid_1's auc: 0.843553
Did not meet early stopping. Best iteration is:
[99]	training's auc: 0.844974	valid_1's auc: 0.843671
Validation AUC:0.8436710040011889
Validation f1 score:78.70018350525483


[I 2021-04-18 14:27:00,084] Trial 1 finished with value: 0.8436710040011889 and parameters: {'learning_rate': 0.010839306644327647, 'colsample_bytree': 0.584899480775861, 'subsample': 0.511910344666861}. Best is trial 1 with value: 0.8436710040011889.
2021/04/18 14:27:00 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of lightgbm. If you encounter errors during autologging, try upgrading / downgrading lightgbm to a supported version, or try upgrading MLflow.


...

Training until validation scores don't improve for 100 rounds
[20]	training's auc: 0.856932	valid_1's auc: 0.855546
[40]	training's auc: 0.890737	valid_1's auc: 0.888823
[60]	training's auc: 0.904689	valid_1's auc: 0.901762
[80]	training's auc: 0.913132	valid_1's auc: 0.909635
[100]	training's auc: 0.920239	valid_1's auc: 0.916397
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.920239	valid_1's auc: 0.916397
Validation AUC:0.916396611715711
Validation f1 score:87.74703083007044


[I 2021-04-18 14:51:53,746] Trial 38 finished with value: 0.916396611715711 and parameters: {'learning_rate': 0.0990172452413256, 'colsample_bytree': 0.7734126989852521, 'subsample': 0.676245554113416}. Best is trial 27 with value: 0.9171468688157753.
2021/04/18 14:51:55 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of lightgbm. If you encounter errors during autologging, try upgrading / downgrading lightgbm to a supported version, or try upgrading MLflow.


Training until validation scores don't improve for 100 rounds
[20]	training's auc: 0.818636	valid_1's auc: 0.816786
[40]	training's auc: 0.819014	valid_1's auc: 0.817724
[60]	training's auc: 0.818987	valid_1's auc: 0.817784
[80]	training's auc: 0.823302	valid_1's auc: 0.822252
[100]	training's auc: 0.825835	valid_1's auc: 0.824951
Did not meet early stopping. Best iteration is:
[99]	training's auc: 0.825931	valid_1's auc: 0.825076
Validation AUC:0.8250757199636066
Validation f1 score:66.23195524719897


[I 2021-04-18 14:52:23,940] Trial 39 finished with value: 0.8250757199636066 and parameters: {'learning_rate': 0.002048925984772501, 'colsample_bytree': 0.7025974840379042, 'subsample': 0.46060154599361697}. Best is trial 27 with value: 0.9171468688157753.


score: 0.9171468688157753
Best hyperparameters: {'learning_rate': 0.09789100804571241, 'colsample_bytree': 0.8204468127501213, 'subsample': 0.7816910577829784}


## Model Validation on test data

In [263]:
# Load best model
lgb_best_model = mlflow.sklearn.load_model("./mlflow-run/5ae87c7ebe0346daac0d0b362d1327c7/artifacts/model")
X_test = X_test[features]

# Make prediction aganist Validation data
lgb_best_val_prediction = lgb_best_model.predict(X_test)

fpr, tpr, thresholds = roc_curve(y_test, lgb_best_val_prediction)
roc_auc = auc(fpr, tpr) # compute area under the curve
print("=====================================")
print("Test AUC:{}".format(roc_auc))
print("=====================================")   

Validation AUC:0.6707697944535749
